<a href="https://colab.research.google.com/github/jmcarbo/medicalimages/blob/main/Pr%C3%A0ctica_5_Imatge_biom%C3%A8dica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import os
import shutil

if not os.path.isfile("bbid.py"):
  os.system("wget https://raw.githubusercontent.com/ostrolucky/Bulk-Bing-Image-downloader/master/bbid.py")
  os.system("chmod +x bbid.py")
shutil.rmtree('images')
os.mkdir("images")
image_classes = ["femur", "pulmonary carcinoma", "enphysema radiography", "melanoma", "erithema nodosum"]
for i in image_classes:
  os.mkdir( "images/" + i)

In [25]:
for i in image_classes:
  os.system(f'./bbid.py -s "{i} dicom" -o "images/{i}" --limit 10')